In [18]:
import numpy as np
from sklearn.linear_model import LogisticRegression 
from  sklearn.metrics import accuracy_score  
from sklearn.model_selection import train_test_split
from sklearn.utils import as_float_array 
import csv 
import math 
import collections 
keys = {'C': 0, 'C#': 1, 'D': 2, 'D#': 3, 'E': 4, 'F': 5,  
      'F#': 6, 'G': 7, 'G#': 8, 'A': 9, 'A#': 10, 'B': 11} 
genreStrings = [] 
allFeaturesMatrix = [] 
allFeaturesDict = {} 
masterGenreDict = {} 
genreArrayDict = {} 
genreToCoefficients = {} 
genreDict = {} 
misclassified = {}
misclassifiedPopSongs = []
topThreePredictions = {}
#songToGenre = {} 

def readMusicData(filename) :
    # open a file
    with open(filename) as f:
        line = csv.reader(f)
        # loop over each row in the file
        rowNum = 0
        for row in line:
            if rowNum == 0 : 
                rowNum += 1
                continue
            # cast each value to a float
            features = []
            currFeature = 0
            nameID = ""
            for value in row :
                if currFeature ==0: 
                    if value not in genreStrings: 
                        genreStrings.append(value) 

                if (currFeature >= 4 and currFeature <= 8) or currFeature == 10 \
                    or currFeature == 11 or currFeature == 13 or currFeature == 14 \
                        or currFeature == 16: 
                        value = float(value)
                if currFeature == 15 : 
                    value = int(value[0])
                if currFeature == 3 : 
                    nameID = str(value)
                if currFeature == 9 : 
                    value = keys[value]
                if currFeature == 12 : 
                    value = 1 if value == "Major" else 0
                features.append(value)
                currFeature += 1
            allFeaturesDict[nameID] = features
            allFeaturesMatrix.append(features)
            rowNum += 1
        # store the row into our dict

def requests(name) :
    songRequests = collections.defaultdict(None)
    count = 0
    with open(name,'r') as file :
        for line in file: 
            if line == "\n":
                break
            line = line.split(' = ')
            name = line[0]
            line[1] = line[1].replace('\"', "")
            ids = line[1].split(", ")
            ids[0] = ids[0].replace('c(', "")
            ids[4] = ids[4].replace(')\n', "")
            songRequests[name] = ids
            count += 1
            if count == 51 : break
    return collections.OrderedDict(sorted(songRequests.items()))
def makeGenreDict(filename, genre, genreArray) : 
    with open(filename) as f: 
        line = csv.reader(f)
        rowNum = 0 
        for row in line: 
            if rowNum == 0 :  
                rowNum += 1 
                continue 
              # cast each value to a float 
              # features = [] 
            currFeature = 0 
            nameID = ""   
            genreY = 0 
            gname = ""  
            for value in row : 
                if currFeature ==0: 
                    gname = value 
                    if value == genre: 
                        genreY = 1 
                if currFeature == 3 :  
                    nameID = str(value) 
                currFeature+=1 
            genreDict[nameID] = str(gname) 
            genreArray.append(genreY) 
            rowNum += 1 
  #extract numerical features only, with the first value as 'id' 
def sigmoid(x): 
    return 1 / (1 + math.exp(-x)) 
def numericalFeaturesOnly(allFeatures, toUse = []) :  
    newFeatures = [] 
    if not toUse : 
        for feature in allFeatures : 
            newFeatures.append(feature[4:]) 
    else :  
        for feature in allFeatures :  
            feature = [feature[i] for i in range(len(feature)) if i in toUse] 
            newFeatures.append(feature) 
    return newFeatures 

def outputResults(toUse = [], average = True) : 
    necessaryDict = {} 
    totalSongs = 0 
    numIncorrect = 0  
  #     print(friends) 
    for person, songIds in friends.items() :  
  #         print(person) 
        songs = [] 
        for song in songIds :  
              #Using all songs 
            info = allFeaturesDict[song] 
            if not toUse :  
                songs.append(info[4:]) 
                necessaryDict[song] = info[4:] 
            else :  
                songs.append([info[i] for i in range(len(info)) if i in toUse]) 
                necessaryDict[song] = [info[i] for i in range(len(info)) if i in toUse] 
#             print(necessaryDict[song]) 
            totalSongs+=1 



        print("Songs for my friend "  + person) 
        print(" Input songs: " ) 
  #         print(songs) 
        for song in songIds :  
            print("\t\t" + allFeaturesDict[song][2] + " by " + allFeaturesDict[song][1])
            scores = [] 
            scoresDict = {} 
  #         for song in songIds: 
#             j=0 
            for i in range(1, len(genreStrings)): 
                score =  as_float_array(necessaryDict[song], copy=True) @ genreToCoefficients[genreStrings[i]][0] 
                sigmoidScore = sigmoid(score + genreToModel[genreStrings[i]].intercept_)                            
                scores.append(sigmoidScore) 
                scoresDict[sigmoidScore] = genreStrings[i] 
            maxScore = max(scores)
            print(scoresDict) 
            print(scoresDict[maxScore]) 
#             print(scores[]) 
            print("Correct genre:"  ) 
            print(genreDict[song]) 
            print(maxScore) 
  #             print(sigmoid(maxScore + genreToModel[scoresDict[maxScore]].intercept_)) 


            if(scoresDict[maxScore]!=genreDict[song]):  
                if(genreDict[song]=='Pop'):
                    misclassifiedPopSongs.append(song)
                if(genreDict[song]=='NULL'): 
                    numIncorrect-=2 
                    totalSongs-=1 
                numIncorrect+=1 
                if scoresDict[maxScore] not in misclassified:
                    misclassified[scoresDict[maxScore]] = collections.defaultdict(int)
#                 if genreDict[song] not in  misclassified[scoresDict[maxScore]]:
#                     misclassified[scoresDict[maxScore]][genreDict[song]] = collections.defaultdict(int)
                misclassified[scoresDict[maxScore]][genreDict[song]]+=1
    print("\n" ) 
    return (numIncorrect, totalSongs) 
def outRes2(x, y, toUse = [], average = True) : 
    necessaryDict = {} 
    totalSongs = 0 
    numIncorrect = 0  
  
    for i in range(len(x)) :  
#             print("\t\t" + allFeaturesDict[song][2] + " by " + allFeaturesDict[song][1])
        scores = [] 
        scoresDict = {} 
#         for song in songIds: 
#             j=0 
        for i in range(1, len(genreStrings)): 
            score =  as_float_array(x[i][1:], copy=True) @ genreToCoefficients[genreStrings[i]][0] 
            sigmoidScore = sigmoid(score + genreToModel[genreStrings[i]].intercept_)                            
            scores.append(sigmoidScore) 
            scoresDict[sigmoidScore] = genreStrings[i] 
        maxScore = max(scores)
        print(scoresDict) 
        print(scoresDict[maxScore]) 
#             print(scores[]) 
        print("Correct genre:"  ) 
        print(genreDict[x[i][0]]) 
        print(maxScore) 
#             print(sigmoid(maxScore + genreToModel[scoresDict[maxScore]].intercept_)) 
        if(scoresDict[maxScore]!=genreDict[x[i][0]]):  
            if(genreDict[x[i][0]]=='NULL'): 
                numIncorrect-=2 
                totalSongs-=1 
            numIncorrect+=1 
            if scoresDict[maxScore] not in misclassified:
                misclassified[scoresDict[maxScore]] = collections.defaultdict(int)
#                 if genreDict[song] not in  misclassified[scoresDict[maxScore]]:
#                     misclassified[scoresDict[maxScore]][genreDict[song]] = collections.defaultdict(int)
            misclassified[scoresDict[maxScore]][y[i]]+=1

#             if(scoresDict[maxScore]!=genreDict[song]):  
#                 if(genreDict[song]=='NULL'): 
#                     numIncorrect-=2 
#                     totalSongs-=1 
#                 numIncorrect+=1 
#                 if scoresDict[maxScore] not in misclassified:
#                     misclassified[scoresDict[maxScore]] = collections.defaultdict(int)
# #                 if genreDict[song] not in  misclassified[scoresDict[maxScore]]:
# #                     misclassified[scoresDict[maxScore]][genreDict[song]] = collections.defaultdict(int)
#                 misclassified[scoresDict[maxScore]][genreDict[song]]+=1
    print("\n" ) 
    return (numIncorrect, totalSongs) 
print("Reading File Data!\n"  ) 
# Our friend song interests.  
friends = requests('SongInput.txt') 
#populate matrix and dict with all features 
readMusicData('SpotifyFeatures.csv') 
print(genreStrings) 
print(len(genreStrings)) 

  #Feature indices, beginning at 4 and ending at 16, inclusive. BTW, 12 features total.  
  #first example: all features 
  # desiredFeatures = [i for i in range(4,17)] 
  #second example: \  Danceability, Speechiness, Acousticness, Instrumentalness, Liveness, Valence, Loudness, and Tempo\   

desiredFeatures = [4,5,8,10,11,13,14,16] 
# desFeat2 = [4, 5, 7, 8, 10, 11, 13, 14, 16] 
desFeat2 = [4, 5, 7, 8, 10, 11, 13, 14, 16] 
# desFeat3 = [4, 5, 7, 8, 13] 
#just the numerical features sans artist/titles/genre/id 
numericalFeatures = numericalFeaturesOnly(allFeaturesMatrix, desFeat2) 
print(numericalFeatures[0])
# model = LogisticRegression() 
# model.fit(allFeaturesMatrix, genreArray) 
# print(allFeaturesMatrix) 
genreToModel = {} 
# genreArrays 
genreToTest = {}
for i in range(1, len(genreStrings)): 
    mdl = LogisticRegression() 
    genreArray = [] 
#     genreDict = {} 
    print(genreStrings[i]) 
    makeGenreDict("SpotifyFeatures.csv" , genreStrings[i], genreArray) 
#     X_train, X_test, y_train, y_test = train_test_split(numericalFeatures, genreArray,test_size=0.005)
#     print(genreArray) 
    fittedmdl = mdl.fit(numericalFeatures, genreArray) 
#     fittedmdl = mdl.fit(X_train, y_train) 
#     genreToTest[genreStrings[i]] = (X_test, y_test)
    print(genreStrings[i]) 
    print(sum(genreArray)) 
#     print(fittedmdl.coef_) 
    floats = as_float_array(fittedmdl.coef_, copy=True) 
    genreToCoefficients[genreStrings[i]] = floats 
    genreArrayDict[genreStrings[i]] = genreArray 
    masterGenreDict[genreStrings[i]] = genreDict 
    genreToModel[genreStrings[i]] = fittedmdl 


  # predicted_classes = model.predict(X) 
  # accuracy = accuracy_score(genreArray.flatten(),predicted_classes) 
  # parameters = model.coef_  



  # print(genreToCoefficients) 


Reading File Data!

['NULL', 'Pop', 'Alternative', 'Blues', 'R&B', 'World', 'Dance', 'Soul', 'Hip-Hop', 'Reggae', 'Country', 'Electronic', 'Rock', 'Comedy', 'Folk', 'Jazz', 'Ska', 'Rap', 'Reggaeton', 'Indie', 'Classical', 'Soundtrack', 'Opera', 'Movie', 'A Capella']
25
[0.502, 0.864, 0.39, 4.82e-05, 0.127, -7.438, 0.0392, 103.948, 0.811]
Pop


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pop
6022
Alternative


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Alternative
5300
Blues


...

Opera
7891
Movie


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Movie
7555
A Capella


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


A Capella
119


In [19]:
numIncorrectClass = 0 
totalClassified = 0 
# print(X_test)
# numInc, totalUpdate = outRes2(X_test, y_test)
numInc, totalUpdate = outputResults(desFeat2) 

totalClassified += totalUpdate 
numIncorrectClass += numInc 
print(numIncorrectClass) 
print(totalClassified) 

# print(numIncorrect) 
print(float(numIncorrectClass)/float(totalClassified))
print(misclassified)
print(misclassifiedPopSongs)


Songs for my friend aarthip
 Input songs: 
		Good as Hell by Lizzo
{0.050243800767749654: 'Pop', 0.04242670648161785: 'Alternative', 0.025854259581594506: 'Blues', 0.024695703433655323: 'R&B', 0.04885152220671571: 'World', 0.10574338348347263: 'Dance', 0.033536095991538575: 'Soul', 0.013797017263380984: 'Hip-Hop', 0.029558255358821346: 'Reggae', 0.05909761525992822: 'Country', 0.07630841667747419: 'Electronic', 0.009879756157225106: 'Rock', 0.009504387935253992: 'Comedy', 0.02766237475770813: 'Folk', 0.03072813964571984: 'Jazz', 0.03283379452951748: 'Ska', 0.022697391363148266: 'Rap', 0.21971186264868808: 'Reggaeton', 0.022685889948632083: 'Indie', 0.004898689111887995: 'Classical', 0.0008863013029678095: 'Soundtrack', 8.177177602340627e-05: 'Opera', 0.015888479918151982: 'Movie', 5.9729667448922103e-05: 'A Capella'}
Reggaeton
Correct genre:
Pop
0.21971186264868808
		Magic In The Hamptons (feat. Lil Yachty) by Social House
{0.11237738485164066: 'Pop', 0.01707015818147825: 'Alternative'

{0.14798888777473984: 'Pop', 0.03339390645340214: 'Alternative', 0.021205911050008117: 'Blues', 0.10173470962524762: 'R&B', 0.013104969360800361: 'World', 0.18610033710468804: 'Dance', 0.07557779860799242: 'Soul', 0.09678478643156892: 'Hip-Hop', 0.24366983954911864: 'Reggae', 0.05730855762505576: 'Country', 0.08131106995405454: 'Electronic', 0.00703302575894285: 'Rock', 0.0005856265946445827: 'Comedy', 0.03658102751739625: 'Folk', 0.06139943600774069: 'Jazz', 0.007602912478204916: 'Ska', 0.17349736503574784: 'Rap', 0.28507472591008404: 'Reggaeton', 0.028126383580775766: 'Indie', 0.0015672773634640388: 'Classical', 0.0010190522468615998: 'Soundtrack', 9.04330107900639e-07: 'Opera', 0.017257624787542326: 'Movie', 0.00012928460181894802: 'A Capella'}
Reggaeton
Correct genre:
NULL
0.28507472591008404
		Gateway Love by Thomas Rhett
{0.06631796306737366: 'Pop', 0.05226436107741421: 'Alternative', 0.03219111386381376: 'Blues', 0.06295141153522803: 'R&B', 0.031748901293829485: 'World', 0.12397

{0.05548476879480958: 'Pop', 0.053439591146047107: 'Alternative', 0.03310761295828619: 'Blues', 0.03351318018551175: 'R&B', 0.0351076793605973: 'World', 0.14521735979086836: 'Dance', 0.04838501179621374: 'Soul', 0.022289194278088915: 'Hip-Hop', 0.15133715070874743: 'Reggae', 0.14529357608220983: 'Country', 0.055921930376564816: 'Electronic', 0.028140556394207376: 'Rock', 0.000560439828866942: 'Comedy', 0.07103467828559336: 'Folk', 0.04261520627092177: 'Jazz', 0.09232314928114028: 'Ska', 0.03504648781578087: 'Rap', 0.15962461285905322: 'Reggaeton', 0.028650869005982814: 'Indie', 0.003382236569761102: 'Classical', 0.0012237693875707715: 'Soundtrack', 2.2336089277911427e-06: 'Opera', 0.015853547923258538: 'Movie', 0.00010842004156794624: 'A Capella'}
Reggaeton
Correct genre:
NULL
0.15962461285905322
Songs for my friend gabea
 Input songs: 
		How Long? by Vampire Weekend
{0.11449936884919476: 'Pop', 0.013254116995611036: 'Alternative', 0.008061994422815284: 'Blues', 0.1113931581452205: 'R&

{0.0660442990703013: 'Pop', 0.014853614065676958: 'Alternative', 0.00895298199834421: 'Blues', 0.06391241002303942: 'R&B', 0.02654879675436556: 'World', 0.03229975451527391: 'Dance', 0.06693844421082075: 'Soul', 0.012497135892420344: 'Hip-Hop', 0.024203320133543437: 'Reggae', 0.050647114694674325: 'Country', 0.0061919009023499085: 'Electronic', 0.004088533905300283: 'Rock', 0.0033530759478713453: 'Comedy', 0.08909512075988389: 'Folk', 0.0674793629283851: 'Jazz', 0.0007109404351348244: 'Ska', 0.018389568701111716: 'Rap', 0.035434582246067006: 'Reggaeton', 0.02478485481167281: 'Indie', 0.017281309473089857: 'Classical', 0.0027456266966024087: 'Soundtrack', 0.005320003840713601: 'Opera', 0.07875726586775457: 'Movie', 0.0014945102306987107: 'A Capella'}
Folk
Correct genre:
Alternative
0.08909512075988389
Songs for my friend nickor
 Input songs: 
		Be with me by Jordy
{0.048540606931951516: 'Pop', 0.03473793924634894: 'Alternative', 0.021144713490644597: 'Blues', 0.047976458764387116: 'R&B'

{0.07000749535894983: 'Pop', 0.06502267377795148: 'Alternative', 0.04083724634686179: 'Blues', 0.03410174665834492: 'R&B', 0.023404589665399392: 'World', 0.16726048273669403: 'Dance', 0.03473786860480675: 'Soul', 0.027654505860860528: 'Hip-Hop', 0.090594328109789: 'Reggae', 0.05681376009386988: 'Country', 0.10985832809983367: 'Electronic', 0.015243516417925667: 'Rock', 0.0020700246467409006: 'Comedy', 0.02142506845643635: 'Folk', 0.028127657140475482: 'Jazz', 0.08736050089708748: 'Ska', 0.04960398287141252: 'Rap', 0.21888120272762732: 'Reggaeton', 0.02409484272388578: 'Indie', 0.0021589134367160944: 'Classical', 0.0010084068476519004: 'Soundtrack', 2.1402354408639483e-06: 'Opera', 0.010392059819068027: 'Movie', 5.143633003805431e-05: 'A Capella'}
Reggaeton
Correct genre:
NULL
0.21888120272762732
		Dance Me A Number by The Steepwater Band
{0.02689637279951741: 'Pop', 0.09094185571290625: 'Alternative', 0.053154618729342934: 'Blues', 0.011549391875015: 'R&B', 0.11380992561894715: 'World'